# Herramienta aplicada unidad 4
## Ejemplo de clasificación de fraudes


Una aplicación común de Machine Learning es la detección de fraudes en transacciones financieras. 
En este notebook, se presenta un ejemplo de cómo implementar una detección de fraudes utilizando algoritmos de Machine Learning en Python.

En esta oportunidad, usaremos el lenguaje Python y algunas de sus librerías más utilizadas. La idea es que conozcas 
cómo se implementan estos modelos, aunque en este caso se usarán datos ficticios y simulados que no representan 
la complejidad de un análisis real en esta área.
    

## Datos

Supongamos que tenemos un conjunto de datos con información sobre transacciones bancarias. Cada observación represnta una transacción y las variables incluidas son:
- Monto de la transacción (en dólares)
- Hora de la transacción (en formato de 24 hrs)
- Tipo de transaccion (retiro, depósito, transferencia)
- Ubicación de la transacción (local, nacional, internacional)
- Tarjeta de crédito utilizada (número de tarjeta)

Para simular los datos, podemos generar un conjunto de observaciones aleatorias con distribuciones específicas para cada variable. En este caso, asumiremos que hay un 5% de transacciones fraudulentas en el conjunto de datos.

En este ejemplo, crearé un conjunto de datos de transacciones financieras con 10,000 transacciones. Estableceré una proporción del % de transacciones fraudulentas, lo que significa que habrá 500 transacciones fraudulentas en el conjunto de datos. Además, crearé algunas características que podrían ser útiles para identificar transacciones fraudulentas, como un monto alto, una ubicación inusual, un
horario inusual y un tipo de transacción inusual.

In [ ]:

# Importar librerías necesarias
import numpy as np
import pandas as pd

# Fijar una semilla para la reproducibilidad
np.random.seed(42)

# Número de transacciones
n_transacciones = 10000

# Crear un conjunto de datos con transacciones normales
datos_normales = pd.DataFrame({
    'monto': np.random.normal(loc=50, scale=10, size=n_transacciones),
    'ubicacion': np.random.choice(['EEUU', 'México', 'Canadá'], size=n_transacciones),
    'horario': np.random.choice(['día', 'noche'], size=n_transacciones),
    'tipo': np.random.choice(['compra', 'retiro'], size=n_transacciones),
    'clase': np.zeros(n_transacciones)
})

# Crear un conjunto de datos con transacciones fraudulentas
datos_fraudulentos = pd.DataFrame({
    'monto': np.random.normal(loc=100, scale=20, size=int(n_transacciones * 0.05)),
    'ubicacion': np.random.choice(['Nigeria', 'Rusia', 'China'], size=int(n_transacciones * 0.05)),
    'horario': np.random.choice(['día', 'noche'], size=int(n_transacciones * 0.05)),
    'tipo': np.random.choice(['compra', 'retiro'], size=int(n_transacciones * 0.05)),
    'clase': np.ones(int(n_transacciones * 0.05))
})

# Combinar los conjuntos de datos y mezclar
datos = pd.concat([datos_normales, datos_fraudulentos]).sample(frac=1, random_state=42).reset_index(drop=True)

# Visualizar las primeras filas de los datos
datos.head()
    


### Explicación de técnicas de modelado para la detección de fraude

Con estos datos simulados, podemos aplicar diferentes técnicas de modelado para detectar patrones en las transacciones 
que puedan indicar la presencia de fraude.

Existen varias técnicas de modelado que se pueden aplicar para detectar patrones de fraude en transacciones, algunas 
de las más comunes son:

1. **Regresión logística**: Un modelo de regresión logística que predice la probabilidad de que una transacción sea 
fraudulenta o no, basado en una serie de variables explicativas.

2. **Árboles de decisión**: Dividen los datos en diferentes subgrupos con características similares, ayudando a 
identificar patrones de fraude y clasificar transacciones.

3. **Redes neuronales**: Modelos basados en el funcionamiento del cerebro humano que identifican patrones complejos 
en grandes conjuntos de datos.

4. **Análisis de anomalías**: Detecta patrones inusuales o anómalos en los datos, útil para identificar transacciones sospechosas.

5. **Clustering**: Agrupa transacciones con características similares, ayudando a detectar patrones de fraude en esos 
grupos.

Para este ejemplo, utilizaremos las técnicas 1, 2 y 5 de manera sintética.
    

### Preparación de los datos

In [ ]:

# Codificar las variables categóricas usando la técnica de codificación one-hot
datos_codificados = pd.get_dummies(datos, columns=['ubicacion', 'horario', 'tipo'])
datos_codificados.head()
    

### Implementación de Regresión Logística

In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report

# Separar los datos en conjuntos de entrenamiento y prueba
X = datos_codificados.drop('clase', axis=1)
y = datos_codificados['clase']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ajustar un modelo de regresión logística
lr_model = LogisticRegression(random_state=42, max_iter=1000)
lr_model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = lr_model.predict(X_test)

# Evaluar el desempeño del modelo en el conjunto de prueba
print("Matriz de confusión:
", confusion_matrix(y_test, y_pred))
print("Reporte de clasificación:
", classification_report(y_test, y_pred))
    

### Predicción con nuevos datos

In [ ]:

# Definir nuevos datos simulados
nuevos_datos = pd.DataFrame({
    'monto': [80, 30, 120, 60, 60, 100],
    'ubicacion': ['EEUU', 'Canadá', 'Rusia', 'México', 'China', 'Nigeria'],
    'horario': ['día', 'noche', 'noche', 'día', 'noche', 'noche'],
    'tipo': ['compra', 'retiro', 'compra', 'retiro', 'retiro', 'retiro']
})

# Preprocesar los nuevos datos
nuevos_datos_procesados = pd.get_dummies(nuevos_datos, columns=['ubicacion', 'horario', 'tipo'])

# Hacer predicciones sobre los nuevos datos
predicciones = lr_model.predict(nuevos_datos_procesados)

# Imprimir las predicciones
print("Predicciones:", predicciones)
    

### Árbol de decisión

In [ ]:

from sklearn.tree import DecisionTreeClassifier

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Crear modelo de árbol de decisión
clf = DecisionTreeClassifier()

# Entrenar el modelo en el conjunto de entrenamiento
clf.fit(X_train, y_train)

# Hacer predicciones en el conjunto de prueba
y_pred = clf.predict(X_test)

# Imprimir la matriz de confusión
print(confusion_matrix(y_test, y_pred))

# Imprimir el informe de clasificación
print(classification_report(y_test, y_pred))
    

### Clustering

In [ ]:

from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Reducir el conjunto de datos para hacer el clustering (sin la variable de clase)
X_clustering = datos_codificados.drop('clase', axis=1)

# Definir el modelo de K-Means con un número arbitrario de clusters
kmeans = KMeans(n_clusters=2, random_state=42)
kmeans.fit(X_clustering)

# Agregar los clusters a los datos originales
datos['cluster'] = kmeans.labels_

# Visualizar la distribución de clases en cada cluster
print(datos.groupby('cluster')['clase'].value_counts())

# Visualización de los clusters (utilizando solo dos dimensiones para simplicidad)
plt.scatter(datos['monto'], datos['cluster'], c=datos['clase'])
plt.xlabel('Monto de la transacción')
plt.ylabel('Cluster asignado')
plt.title('Distribución de clusters')
plt.show()
    